In [ ]:
import os
import sys
import torch
import pandas as pd
import numpy as np

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from hd_epic_utils import get_hd_epic, load_videos

In [ ]:
frames = 10
HD_EPIC_PATH = '<HD-EPIC-ANNOTATIONS-PATH>'
VIDEO_PATH = f'<HD-EPIC-VIDEO-PATH>'
SUBSET_PATH = '../subsets/hd-epic.csv'

qa_data, qa_answers = get_hd_epic(HD_EPIC_PATH)

q_uids = pd.read_csv(SUBSET_PATH)['q_uid'].to_list()

if not os.path.exists(f"hd_epic_features_{frames}"):
    os.mkdir(f"hd_epic_features_{frames}")

In [ ]:
import math
import numpy as np
import torch.nn.functional as F
from tqdm import tqdm
from extract.preprocessing import Preprocessing
from args import MODEL_DIR
import clip

In [ ]:
preprocess = Preprocessing()
model, _ = clip.load("ViT-L/14", download_root=MODEL_DIR)
model.eval()
model = model.cuda()

In [ ]:
with torch.no_grad():
    for q_uid in q_uids:
        video_frames, input_keys, stride, frame_idx_dict = load_videos(qa_data[q_uid], frames, 1, VIDEO_PATH, 224, 224)
        video = torch.from_numpy(np.concatenate(video_frames)).permute((0, 3, 1, 2))
        video = preprocess(video)
        video = video.cuda()
        features = model.encode_image(video)
        features = features.cpu().numpy().astype('float16')
        np.save(f'hd_epic_features_{frames}/{q_uid}.npy', {'features': features, 'stride': stride}, allow_pickle=True)